In [1]:
import pyspark
from pyspark import SparkContext
from pyspark.sql import SparkSession

In [2]:
spark = SparkSession.builder.appName("SparkOperations").getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/08/12 09:26:13 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [3]:
lst = [[i] for i in range(1, 100, 3)]
df = spark.createDataFrame(lst, ["num"])
df.show()

+---+
|num|
+---+
|  1|
|  4|
|  7|
| 10|
| 13|
| 16|
| 19|
| 22|
| 25|
| 28|
| 31|
| 34|
| 37|
| 40|
| 43|
| 46|
| 49|
| 52|
| 55|
| 58|
+---+
only showing top 20 rows



In [4]:
data = [
    ("Alice", 30, 3000),
    ("Bob", 45, 4000),
    ("Charlie", 35, 3500),
    ("David", 50, 5000),
    ("Eve", 29, 2800)
]
df = spark.createDataFrame(data, ["name", "age", "salary"])
df.show()

+-------+---+------+
|   name|age|salary|
+-------+---+------+
|  Alice| 30|  3000|
|    Bob| 45|  4000|
|Charlie| 35|  3500|
|  David| 50|  5000|
|    Eve| 29|  2800|
+-------+---+------+



In [5]:
# Filter rows where age is greater than 30
df_filtered = df.filter(df["age"] > 30)

# Show the filtered DataFrame
print("Filtered DataFrame (age > 30):")
df_filtered.show()

Filtered DataFrame (age > 30):
+-------+---+------+
|   name|age|salary|
+-------+---+------+
|    Bob| 45|  4000|
|Charlie| 35|  3500|
|  David| 50|  5000|
+-------+---+------+



In [6]:
from pyspark.sql.functions import col, when

df_with_column = df.withColumn(
    "salary_increase",
    when(col("salary") < 4000, "Low")
    .otherwise("High")
)

df_with_column.show()

+-------+---+------+---------------+
|   name|age|salary|salary_increase|
+-------+---+------+---------------+
|  Alice| 30|  3000|            Low|
|    Bob| 45|  4000|           High|
|Charlie| 35|  3500|            Low|
|  David| 50|  5000|           High|
|    Eve| 29|  2800|            Low|
+-------+---+------+---------------+



In [7]:
print(df.count())
df.show()

5
+-------+---+------+
|   name|age|salary|
+-------+---+------+
|  Alice| 30|  3000|
|    Bob| 45|  4000|
|Charlie| 35|  3500|
|  David| 50|  5000|
|    Eve| 29|  2800|
+-------+---+------+



In [8]:
df.agg({'salary': 'avg'}).show()
df.agg({'salary': 'sum'}).show()

+-----------+
|avg(salary)|
+-----------+
|     3660.0|
+-----------+

+-----------+
|sum(salary)|
+-----------+
|      18300|
+-----------+



In [9]:
df_with_column.write \
    .format("csv") \
    .option("header", "true") \
    .mode("overwrite") \
    .save("./files/")

In [12]:
from pyspark.sql.functions import split, explode
df = spark.read.csv('finance.csv', header=True, inferSchema=True)
words_df = df.select(explode(split(col("Series_title_1"), " ")).alias("word"))
words_count = words_df.groupBy("word").count()
words_count.show()

+-----------+-----+
|       word|count|
+-----------+-----+
|   Salaries| 1265|
|      Sales| 3840|
|    income)| 3840|
|  Operating| 1265|
|      wages| 1265|
|  operating| 1265|
|expenditure| 1265|
|        and| 2530|
|  Purchases| 1265|
| (operating| 3840|
|     profit| 1265|
+-----------+-----+

